In [0]:
dbutils.widgets.dropdown("env_stage", "dev", ["dev", "prod", "qa","dev_synxis_2_0", "prod_synxis_2_0", "qa_synxis_2_0"], "Pipeline stage")
dbutils.widgets.dropdown("is_usd_currency", "True", ["True", "False"], "Use USD currency")
dbutils.widgets.text("eval_days", "30", "Evaluation Window (Days)")
dbutils.widgets.dropdown("plot_hotels", "False", ["True", "False"], "Display Triangular Plots")

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import os
from datetime import date
from datetime import datetime, timedelta,date
from phgml.data.config import ModuleConfig
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import plotly.express as px
import plotly.graph_objects as go
import re
from datetime import datetime, timedelta
sns.set()

from phgml.data.config import ForecastingHotelConfigProvider,EnvironmentConfig
from phgml.data.processing_distr_spark import preprocess_data,get_running_total
from phgml.utilities.task_utilities import str_to_lst, str_to_bool
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error

In [0]:
params = {}
params["ENV"] = dbutils.widgets.get("env_stage")
params["IS_USD_CURRENCY"] = str_to_bool(getArgument("is_usd_currency"))
params["PLOT_HOTELS"] = str_to_bool(getArgument("plot_hotels"))
params["EVAL_DAYS"] = int(dbutils.widgets.get("eval_days"))
params["REVENUE_COL"] = "_reservationRevenuePerRoomUSD"
params["ROOMS_COL"] = "_rooms"

env_config_rv = EnvironmentConfig(env=params["ENV"], target="REVENUE", spark=spark, is_usd_currency=params["IS_USD_CURRENCY"])
env_config_rm = EnvironmentConfig(env=params["ENV"], target="ROOMS", spark=spark, is_usd_currency=params["IS_USD_CURRENCY"])

In [0]:
params["INFERENCE_SUMMARY_TABLE"] = env_config_rv.inference_summary_table
params["INFERENCE_TRIANGULAR_METRIC_TABLE"] = env_config_rv.inference_triangular_metric_table

In [0]:
hotel_ids = spark.sql(
    "select distinct HotelID,HotelName,No_of_rooms from dev_data.dim_hotels_data"
).toPandas()
hotel_ids = hotel_ids.sort_values("HotelID")
hotel_ids["HotelName"] = hotel_ids["HotelName"].apply(lambda x: x[:20])

In [0]:
# Pick evel period
eval_period_end = datetime.now() - timedelta(days=2)
eval_period_start = eval_period_end - timedelta(days=params["EVAL_DAYS"])
print(f"Evaluate from {eval_period_start} to {eval_period_end}")

In [0]:
# Load actuals
if "synxis_2_0" in params['ENV']:
    dfsp_src = spark.sql(
        f"select * from {env_config_rv.source_data_table}"
        )
    
    dfsp_src = dfsp_src.withColumn(
        'cancellationDate',
        F.when(F.col('status') == 'No-show', F.col('_StayDates')).otherwise(F.col('cancellationDate'))
    )
else:
    dfsp_src = spark.sql(
        f"select a.TransactionID,a.HotelID,a._StayDates,a.confirmationDate,a.departureDate,a.channel,a.status,a.cancellationNumber,a._reservationRevenuePerRoomUSD,a._rooms,b.cancellationDate from {env_config.source_data_table} as a left join {env_config.transaction_data_table} as b on a.TransactionID=b.TransactionID"
        )
    
# load booking statuses 
result = spark.sql("SELECT * FROM phg_data.bookings_status")

confirmed_status_list = [row['status'] for row in result.filter(result.scenario == 'confirmed').collect()]
cancelled_status_list = [row['status'] for row in result.filter(result.scenario == 'cancelled').collect()]

# Display the list
print(f"Confirmed Booking Status List: {confirmed_status_list}")
print(f"Cancelled Booking Status List: {cancelled_status_list}")

columns = [
    "HotelID",
    "_StayDates",
    "confirmationDate",
    "channel",
    "status",
    params["REVENUE_COL"],
    params["ROOMS_COL"],
]

dfsp = dfsp_src.filter(
        (F.col('status').isin(confirmed_status_list)) & (dfsp_src.cancellationDate.isNull())
    ).select(columns)

actuals_inc = preprocess_data(
    dfsp,
    False,
    params["REVENUE_COL"],
    params["ROOMS_COL"],
    pd.to_datetime(eval_period_start),
    cancel_aware=False
)

actuals_exc = preprocess_data(
    dfsp,
    False,
    params["REVENUE_COL"],
    params["ROOMS_COL"],
    pd.to_datetime(eval_period_start),
    cancel_aware=False
)

window_spec = Window.partitionBy("HotelID", "_StayDates", "confirmationDate")

actuals_inc = actuals_inc\
    .withColumn("_reservationRevenuePerRoomUSD", F.sum("_reservationRevenuePerRoomUSD").over(window_spec))\
    .withColumn("_rooms", F.sum("_rooms").over(window_spec))
actuals_inc = actuals_inc.dropDuplicates(["HotelID", "_StayDates", "confirmationDate"])

actuals_exc = actuals_exc\
    .withColumn("_reservationRevenuePerRoomUSD", F.sum("_reservationRevenuePerRoomUSD").over(window_spec))\
    .withColumn("_rooms", F.sum("_rooms").over(window_spec))
actuals_exc = actuals_exc.dropDuplicates(["HotelID", "_StayDates", "confirmationDate"])

rv_actuals_inc = get_running_total(actuals_inc, params["REVENUE_COL"], cancel_aware=False)
rv_actuals_inc = rv_actuals_inc.withColumn("pms_sync_off", F.lit(False))
rv_actuals_exc = get_running_total(actuals_exc, params["REVENUE_COL"], cancel_aware=False)
rv_actuals_exc = rv_actuals_exc.withColumn("pms_sync_off", F.lit(True))

rm_actuals_inc = get_running_total(actuals_inc, params["ROOMS_COL"], cancel_aware=False)
rm_actuals_inc = rm_actuals_inc.withColumn("pms_sync_off", F.lit(False))
rm_actuals_exc = get_running_total(actuals_exc, params["ROOMS_COL"], cancel_aware=False)
rm_actuals_exc = rm_actuals_exc.withColumn("pms_sync_off", F.lit(True))

In [0]:
# Concat pms_off and pms_on dataframes
rv_actuals_df = rv_actuals_inc.union(rv_actuals_exc)
rm_actuals_df = rm_actuals_inc.union(rm_actuals_exc)
req_cols =['HotelID', '_StayDates', 'confirmationDate','booking_lead','cum_sum_value','pms_sync_off']
rv_actuals_df = rv_actuals_df.select(req_cols).withColumnRenamed("cum_sum_value", "revenue")
rm_actuals_df = rm_actuals_df.select(req_cols).withColumnRenamed("cum_sum_value", "rooms")

del actuals_inc, actuals_exc, rv_actuals_inc, rv_actuals_exc, rm_actuals_inc, rm_actuals_exc

In [0]:
# Load inferences - Revenue
cols = ['HotelID', 'stay_date', 'booking_date', 'timestamp', 'pms_sync_off', 'day_index', 'y_med']
inference_rv = spark.sql(f"SELECT * FROM {env_config_rv.inference_output_table}")
inference_rv = inference_rv.filter(F.col("stay_date") >= F.lit(eval_period_start).cast("date"))

grouped_df = (
    inference_rv.groupBy("pms_sync_off", "timestamp", "HotelID", "stay_date")
    .agg(F.min("booking_date").alias("eval_start_date"))
)

# Join the eval_start_date back to the original DataFrame
inference_rv = inference_rv.join(
    grouped_df,
    on=["pms_sync_off", "timestamp", "HotelID", "stay_date"],
    how="inner"
)

inference_rv = inference_rv\
    .filter(F.col("booking_date") > F.col("eval_start_date"))\
    .withColumn("day_ahead_index",F.datediff(F.col("stay_date"), F.col("eval_start_date")))\
    .withColumn("booking_index",F.col("day_index"))

inference_rv = inference_rv.withColumnRenamed("booking_date", "confirmationDate")
inference_rv = inference_rv.withColumnRenamed("y_med", "y_pred")
inference_rv = inference_rv.withColumnRenamed("stay_date", "_StayDates")
inference_rv = inference_rv.withColumn("_StayDates", F.to_date(F.col("_StayDates")))
inference_rv = inference_rv.withColumn("confirmationDate", F.to_date(F.col("confirmationDate")))

cols = ['HotelID', '_StayDates', 'confirmationDate', "day_ahead_index", "booking_index",'y_pred', 'pms_sync_off', 'eval_start_date', 'timestamp']
inference_rv = inference_rv.select(cols)

# Merge actuals - Revenue
inference_rv=inference_rv.join(rv_actuals_df, on= ["HotelID", "_StayDates", "confirmationDate", "pms_sync_off"], how="left")
inference_rv = inference_rv.withColumnRenamed("revenue", "y_true")
inference_rv = inference_rv.filter((F.col("y_true").isNotNull()) & (F.col("y_true") != 0))

req_cols = ['HotelID', '_StayDates', 'confirmationDate', "day_ahead_index", "booking_index",'y_pred', 'y_true', 'pms_sync_off','timestamp']
inference_rv = inference_rv.select(req_cols)

inference_rv = inference_rv.withColumn(
        "SAPE",
        F.abs(F.col("y_true") - F.col("y_pred"))
        * 2
        / (F.abs(F.col("y_true")) + F.abs(F.col("y_pred"))),
    )
inference_rv = inference_rv.withColumn("target_type", F.lit("REVENUE"))

In [0]:
# Load inferences - Rooms
cols = ['HotelID', 'stay_date', 'booking_date', 'timestamp', 'pms_sync_off', 'day_index', 'y_med']
inference_rm = spark.sql(f"SELECT * FROM {env_config_rm.inference_output_table}")
inference_rm = inference_rm.filter(F.col("stay_date") >= F.lit(eval_period_start).cast("date"))

grouped_df = (
    inference_rm.groupBy("pms_sync_off", "timestamp", "HotelID", "stay_date")
    .agg(F.min("booking_date").alias("eval_start_date"))
)

# Join the eval_start_date back to the original DataFrame
inference_rm = inference_rm.join(
    grouped_df,
    on=["pms_sync_off", "timestamp", "HotelID", "stay_date"],
    how="inner"
)

inference_rm = inference_rm\
    .filter(F.col("booking_date") > F.col("eval_start_date"))\
    .withColumn("day_ahead_index",F.datediff(F.col("stay_date"), F.col("eval_start_date")))\
    .withColumn("booking_index",F.col("day_index"))

inference_rm = inference_rm.withColumnRenamed("booking_date", "confirmationDate")
inference_rm = inference_rm.withColumnRenamed("y_med", "y_pred")
inference_rm = inference_rm.withColumnRenamed("stay_date", "_StayDates")
inference_rm = inference_rm.withColumn("_StayDates", F.to_date(F.col("_StayDates")))
inference_rm = inference_rm.withColumn("confirmationDate", F.to_date(F.col("confirmationDate")))

cols = ['HotelID', '_StayDates', 'confirmationDate', "day_ahead_index", "booking_index",'y_pred', 'pms_sync_off', 'eval_start_date', 'timestamp']
inference_rm = inference_rm.select(cols)

# Merge actuals - Rooms
inference_rm = inference_rm.join(rm_actuals_df, on= ["HotelID", "_StayDates", "confirmationDate", "pms_sync_off"], how="left")
inference_rm = inference_rm.withColumnRenamed("rooms", "y_true")
inference_rm = inference_rm.filter((F.col("y_true").isNotNull()) & (F.col("y_true") != 0))

req_cols = ['HotelID', '_StayDates', 'confirmationDate', "day_ahead_index", "booking_index",'y_pred', 'y_true', 'pms_sync_off','timestamp']
inference_rm = inference_rm.select(req_cols)

inference_rm = inference_rm.withColumn(
        "SAPE",
        F.abs(F.col("y_true") - F.col("y_pred"))
        * 2
        / (F.abs(F.col("y_true")) + F.abs(F.col("y_pred"))),
    )
inference_rm = inference_rm.withColumn("target_type", F.lit("ROOMS"))

In [0]:
output_df = inference_rv.union(inference_rm)
output_df = output_df.withColumn("eval_start_date", F.lit(eval_period_start))
output_df = output_df.withColumn("eval_end_date", F.lit(eval_period_end))
output_df = output_df.withColumn("Timestamp", F.current_timestamp())

In [0]:
(
    output_df.write.mode("append") # "overwrite", "append"
    .option("overwriteSchema", "true")    
    .saveAsTable(params["INFERENCE_SUMMARY_TABLE"])
)
del inference_rv, inference_rm

In [0]:
grouped_df = output_df.groupBy(
    "pms_sync_off", "target_type", "HotelID", "day_ahead_index", "booking_index"
).agg(*[F.avg(c).alias(c) for c in output_df.columns if 'SAPE' in c])

mean_sapes_df = grouped_df.select(
    "pms_sync_off", "target_type", "HotelID", "day_ahead_index", "booking_index",
    *[c for c in grouped_df.columns if 'SAPE' in c]
)

melted_mean_sapes_df = mean_sapes_df.selectExpr(
    "pms_sync_off", 
    "target_type", 
    "HotelID", 
    "day_ahead_index", 
    "booking_index", 
    "stack(" +
        f"{len([c for c in mean_sapes_df.columns if 'SAPE' in c])}, " +
        ", ".join([f"'{c}', {c}" for c in mean_sapes_df.columns if 'SAPE' in c]) +
    ") as (variable, value)"
)

melted_mean_sapes_df = melted_mean_sapes_df\
    .withColumn("eval_start_date", F.lit(eval_period_start))\
    .withColumn("eval_end_date", F.lit(eval_period_end))\
    .withColumn("Timestamp", F.current_timestamp())

In [0]:
(
    melted_mean_sapes_df.write.mode("append") # "overwrite", "append"
    .option("overwriteSchema", "true")    
    .saveAsTable(params["INFERENCE_TRIANGULAR_METRIC_TABLE"])
)

In [0]:
# Define a custom colormap with red (low), yellow (middle), and green (high)
if params["PLOT_HOTELS"]:
    melted_mean_sapes_df_pd = melted_mean_sapes_df.toPandas()
    colors = [(0, "green"), (0.40, "yellow"),(1, "red")]
    custom_palette = LinearSegmentedColormap.from_list("custom_palette", colors)

    target_df = melted_mean_sapes_df_pd[(melted_mean_sapes_df_pd.pms_sync_off==False) & (melted_mean_sapes_df_pd.target_type=='REVENUE')]

    for hid in target_df.HotelID.unique():
        check_triangle = target_df[(target_df.HotelID==hid)]

        VMAX=check_triangle.groupby(['HotelID']).agg({'value':['min','max']}).reset_index()[('value','max')][0]
        VMIN=check_triangle.groupby(['HotelID']).agg({'value':['min','max']}).reset_index()[('value','min')][0]

        different_plots = sorted(check_triangle.variable.unique())
        total_plots = len(different_plots)#len([i for i in range(1,DAY_AHEAD+1)])
        plots_per_row = target_df.variable.nunique()
        rows_needed = int(np.ceil(total_plots/plots_per_row))

        pointer = 0
        for row in range(rows_needed):
            fig,ax = plt.subplots(nrows=1,ncols=plots_per_row, figsize=(30,6))
            for col in range(plots_per_row):
                if pointer<total_plots:
                    check_variant = check_triangle[check_triangle.variable==different_plots[pointer]]
                    df_plot_inv = check_variant[['day_ahead_index','booking_index','value']].pivot(index='booking_index', columns='day_ahead_index', values='value')
                    df_plot_inv = df_plot_inv.sort_index(ascending=False)
                    sns.heatmap(df_plot_inv,cmap=custom_palette, vmin=VMIN, vmax=VMAX)
                    ax.title.set_text(f' Hotel: {hid} | variant: {different_plots[pointer]}')
                    pointer+=1